In [1]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1.
nH = 8
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="ccpvdz", verbose=4)
mf = scf.RHF(mol).density_fit()
mf.kernel()

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()
print(mycc.e_corr)

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-35-generic', version='#35~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Oct 14 13:55:17 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Thu Nov 20 18:34:50 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 8
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      1.000000000000   0.000000000000  

In [4]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 5,
            'n_blocks': 10,
            'n_walkers': 30,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'cisd_ad',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [ ]:
# import pickle
# from pyscf.ci.cisd import CISD
# from pyscf.cc.ccsd import CCSD
# from pyscf.cc.uccsd import UCCSD
# from pyscf import lib, ao2mo, df, mcscf
# from ad_afqmc import pyscf_interface

# def prep_lnoafqmc(mf_cc,mo_coeff,options,norb_act,nelec_act,
#                   prjlo=[],norb_frozen=[],ci1=None,ci2=None,chol_cut=1e-5,
#                   option_file='options.bin',
#                   mo_file="mo_coeff.npz",
#                   amp_file="amplitudes.npz",
#                   chol_file="FCIDUMP_chol"):
    
    
#     with open(option_file, 'wb') as f:
#         pickle.dump(options, f)
    
#     if isinstance(mf_cc, (CCSD, UCCSD)):
#         mf = mf_cc._scf
#     else:
#         mf = mf_cc
    
#     ci2 = ci2.transpose(0, 2, 1, 3)

#     if options["trial"] == "cisd":
#         np.savez(amp_file, ci1=ci1, ci2=ci2)

#     mol = mf.mol

#     if isinstance(norb_frozen, (int, float)) and norb_frozen == 0:
#         norb_frozen = []
#     elif isinstance(norb_frozen, int):
#         norb_frozen = np.arange(norb_frozen)
#     act_idx = np.array([i for i in range(mol.nao) if i not in norb_frozen])
    
#     print('\n')
#     print('# Generating Cholesky Integrals')
#     mc = mcscf.CASSCF(mf, norb_act, nelec_act)
#     mc.frozen = norb_frozen
#     nelec = mc.nelecas
#     mc.mo_coeff = mo_coeff
#     h1e, enuc = mc.get_h1eff()
#     nbasis = h1e.shape[-1]

#     if getattr(mf, "with_df", None) is not None:
#         print("# Composing AO ERIs from DF basis")
#         chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
#         chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
#         chol_df = chol_df.reshape((-1, mol.nao, mol.nao))
#         eri_ao_df = lib.einsum('lpq,lrs->pqrs', chol_df, chol_df)
#         print("# Composing active space MO ERIs from AO ERIs")
#         eri_mo_df = ao2mo.kernel(eri_ao_df,mo_coeff[:,act_idx],compact=False)
#         eri_mo_df = eri_mo_df.reshape(nbasis**2,nbasis**2)
#         print("# Decomposing MO ERIs to Cholesky vectors")
#         print(f"# Cholesky cutoff is: {chol_cut}")
#         chol = pyscf_interface.modified_cholesky(eri_mo_df,max_error=chol_cut)
#     else:
#         eri_mo = ao2mo.kernel(mf.mol,mo_coeff[:,act_idx],compact=False)
#         chol = pyscf_interface.modified_cholesky(eri_mo,max_error=chol_cut)
    
#     print("# Finished calculating Cholesky integrals")
#     print('# Size of the correlation space')
#     print(f'# Number of electrons: {nelec}')
#     print(f'# Number of basis functions: {nbasis}')
#     print(f'# Number of Cholesky vectors: {chol.shape[0]}')
    
#     chol = chol.reshape((-1, nbasis, nbasis))
#     v0 = 0.5 * lib.einsum('nik,njk->ij', chol, chol, optimize='optimal')
#     h1e_mod = h1e - v0
#     chol = chol.reshape((chol.shape[0], -1))

#     np.savez(mo_file,prjlo=prjlo)

#     pyscf_interface.write_dqmc(
#         h1e,
#         h1e_mod,
#         chol,
#         sum(nelec),
#         nbasis,
#         enuc,
#         ms=mol.spin,
#         filename=chol_file,
#     )
    
#     return None

In [5]:
from pyscf.ci.cisd import CISD
from pyscf.cc.ccsd import CCSD
from pyscf import lib
from ad_afqmc.lno.cc import LNOCCSD
from ad_afqmc.lno_afqmc import lno_maker, lno_afqmc
from ad_afqmc.lno.base import lno

frozen = nfrozen
thresh = 1e-4
chol_cut = 1e-8
eris = None
run_frg_list = [0]

mfcc = mf

if isinstance(mfcc, (CCSD, CISD)):
    full_cisd = True
    lnomf = mfcc._scf
else:
    full_cisd = False
    lnomf = mfcc

if isinstance(thresh, list):
    thresh_occ, thresh_vir = thresh
else:
    thresh_occ = thresh*10
    thresh_vir = thresh

lno_cc = LNOCCSD(lnomf, thresh=thresh, frozen=frozen)
lno_cc.thresh_occ = thresh_occ
lno_cc.thresh_vir = thresh_vir
lno_cc.lo_type = 'boys'
lno_cc.no_type = 'ie'
no_type = 'ie'
lno_cc.frag_lolist = '1o'
lno_cc.force_outcore_ao2mo = True

s1e = lnomf.get_ovlp()
lococc = lno_cc.get_lo(lo_type='boys') # localized active occ orbitals
# lococc,locvir = lno_maker.get_lo(lno_cc,lo_type) ### fix this for DF
if eris is None: eris = lno_cc.ao2mo()

frag_lolist = [[i] for i in range(lococc.shape[1])]
print(frag_lolist)
nfrag = len(frag_lolist)

frozen_mask = lno_cc.get_frozen_mask()
thresh_pno = [thresh_occ,thresh_vir]
print(f'# lno thresh {thresh_pno}')

if run_frg_list is None:
    run_frg_list = range(nfrag)

frag_nonvlist = None
if frag_nonvlist is None: frag_nonvlist = lno_cc.frag_nonvlist
if frag_nonvlist is None: frag_nonvlist = [[None,None]] * nfrag

eorb_cc = np.empty(nfrag,dtype='float64')
    
from jax import random
seeds = random.randint(random.PRNGKey(options["seed"]),
                    shape=(len(run_frg_list),), minval=0, maxval=100*nfrag)

for ifrag in run_frg_list:
    print(f'\n########### running fragment {ifrag+1} ##########')

    fraglo = frag_lolist[ifrag]
    orbfragloc = lococc[:,fraglo]
    THRESH_INTERNAL = 1e-10
    frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
    frzfrag, orbfrag, can_orbfrag \
         = lno.make_fpno1(lno_cc, eris, orbfragloc, no_type,
                            THRESH_INTERNAL, thresh_pno,
                            frozen_mask=frozen_mask,
                            frag_target_nocc=None,
                            frag_target_nvir=None,
                            canonicalize=True)

    mol = mf.mol
    nocc = mol.nelectron // 2 
    nao = mol.nao
    actfrag = np.array([i for i in range(nao) if i not in frzfrag])
    # frzocc = np.array([i for i in range(nocc) if i in frzfrag])
    actocc = np.array([i for i in range(nocc) if i in actfrag])
    actvir = np.array([i for i in range(nocc,nao) if i in actfrag])
    nactocc = len(actocc)
    nactocc = len(actocc)
    nactvir = len(actvir)
    prjlo = orbfragloc.T @ s1e @ orbfrag[:,actocc]
    nelec_act = nactocc*2
    norb_act = nactocc+nactvir

    print(f'# active orbitals: {actfrag}')
    print(f'# active occupied orbitals: {actocc}')
    print(f'# active virtual orbitals: {actvir}')
    print(f'# frozen orbitals: {frzfrag}')
    print(f'# number of active electrons: {nelec_act}')
    print(f'# number of active orbitals: {norb_act}')
    print(f'# number of frozen orbitals: {len(frzfrag)}')

    # mp2 is not invariant to lno transformation
    # needs to be done in canoical HF orbitals
    # which the globel mp2 is calculated in
    print('# running fragment MP2')
    ecorr_p2 = \
        lno_maker.lno_mp2_frg_e(lnomf,frzfrag,orbfragloc,can_orbfrag)
    ecorr_p2 = f'{ecorr_p2:.8f}'
    print(f'# LNO-MP2 Orbital Energy: {ecorr_p2}')
    
    print('# running fragment CCSD')
    mcc, ecorr_cc = \
        lno_maker.lno_cc_solver(lnomf,orbfrag,orbfragloc,frozen=frzfrag)
    ecorr_cc = f'{ecorr_cc:.8f}'
    print(f'# LNO-CCSD Energy: {mcc.e_tot}')
    print(f'# LNO-CCSD Orbital Energy: {ecorr_cc}')

    ci1 = np.array(mcc.t1)
    ci2 = mcc.t2 + lib.einsum("ia,jb->ijab",ci1,ci1)

    options["seed"] = seeds[ifrag]
    lno_afqmc.prep_lnoafqmc(
        mf,orbfrag,options,
        norb_act=norb_act,nelec_act=nelec_act,
        prjlo=prjlo,norb_frozen=frzfrag,
        ci1=ci1,ci2=ci2,chol_cut=chol_cut,
        )
    lno_afqmc.run_afqmc(options)


lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 19.005131246717  delta_f= 19.0051  |g|= 8.24284  1 KF 3 Hx
macro= 2  f(x)= 18.430068362526  delta_f= -0.575063  |g|= 7.35653  1 KF 3 Hx
macro= 3  f(x)= 17.929527568121  delta_f= -0.500541  |g|= 6.49826  1 KF 3 Hx
macro= 4  f(x)= 17.510904508772  delta_f= -0.418623  |g|= 5.54766  1 KF 3 Hx
macro= 5  f(x)= 17.180436592219  delta_f= -0.330468  |g|= 4.51773  1 KF 3 Hx
macro= 6  f(x)= 16.943099158349  delta_f= -0.237337  |g|= 3.42266  1 KF 3 Hx
macro= 7  f(x)= 16.80252356881  delta_f= -0.140576  |g|= 2.27756  1 KF 3 Hx
macro= 8  f(x)= 16.760599364979  delta_f= -0.0419242  |g|= 1.0982  1 KF 3 Hx
mac

Traceback (most recent call last):
  File "/home/sharmagroup/sharmagroup/cs_afqmc/ad_afqmc/lno_afqmc/lno_afqmc_cisd/run_lnoafqmc.py", line 43, in <module>
    ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: Non-hashable static arguments are not supported. An error occurred while trying to hash an object of type <class 'ad_afqmc.lno_afqmc.wavefunctions.cisd_ad'>, cisd_ad(norb=14, nelec=(3, 3), n_opt_iter=30, n_batch=1). The error was:
TypeError: unhashable type: 'cisd_ad'



In [5]:
import h5py
from ad_afqmc import config
from functools import partial
from ad_afqmc import hamiltonian
from ad_afqmc.prop_unrestricted import wavefunctions, propagation, sampling
from jax import numpy as jnp
print = partial(print, flush=True)

def _prep_afqmc(option_file="options.bin",
                mo_file="mo_coeff.npz",
                amp_file="amplitudes.npz",
                chol_file="FCIDUMP_chol"):
    
    try:
        with open(option_file, "rb") as f:
            options = pickle.load(f)
    except:
        print('# Using default options')
        options = {}

    options["dt"] = options.get("dt", 0.01)
    options["n_exp_terms"] = options.get("n_exp_terms",6)
    options["n_walkers"] = options.get("n_walkers", 50)
    options["n_prop_steps"] = options.get("n_prop_steps", 50)
    options["n_ene_blocks"] = options.get("n_ene_blocks", 10)
    options["n_sr_blocks"] = options.get("n_sr_blocks", 5)
    options["n_blocks"] = options.get("n_blocks", 50)
    options["seed"] = options.get("seed", np.random.randint(1, int(1e6)))
    options["n_eql"] = options.get("n_eql", 1)
    options["ad_mode"] = options.get("ad_mode", None)
    assert options["ad_mode"] in [None, "forward", "reverse", "2rdm"]
    options["orbital_rotation"] = options.get("orbital_rotation", True)
    options["do_sr"] = options.get("do_sr", True)
    options["walker_type"] = options.get("walker_type", "rhf")
    options["symmetry"] = options.get("symmetry", False)
    options["save_walkers"] = options.get("save_walkers", False)
    options["trial"] = options.get("trial", None)
    options["ene0"] = options.get("ene0", 0.0)
    options["free_projection"] = options.get("free_projection", False)
    options["n_batch"] = options.get("n_batch", 1)

    if options['use_gpu']:
        config.afqmc_config["use_gpu"] = True

    config.setup_jax()
    MPI = config.setup_comm()
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()

    if rank == 0:
        print(f"# Number of MPI ranks: {size}\n#")

    trial = options['trial']
    if 'u' not in trial.lower():
        with h5py.File(chol_file, "r") as fh5:
            [nelec, nmo, ms, nchol] = fh5["header"]
            h0 = jnp.array(fh5.get("energy_core"))
            h1 = jnp.array(fh5.get("hcore")).reshape(nmo, nmo)
            chol = jnp.array(fh5.get("chol")).reshape(-1, nmo, nmo)
            h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(nmo, nmo)
    elif 'u' in trial.lower():
        with h5py.File(chol_file, "r") as fh5:
            [nelec, nmo, ms, nchol] = fh5["header"]
            h0 = jnp.array(fh5.get("energy_core"))
            h1 = jnp.array(fh5.get("hcore")).reshape(2, nmo, nmo)
            chol = jnp.array(fh5.get("chol")).reshape(2, -1, nmo, nmo)
            h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(2, nmo, nmo)

    assert type(ms) is np.int64
    assert type(nelec) is np.int64
    assert type(nmo) is np.int64
    assert type(nchol) is np.int64
    ms, nelec, nmo, nchol = int(ms), int(nelec), int(nmo), int(nchol)
    nelec_sp = ((nelec + abs(ms)) // 2, (nelec - abs(ms)) // 2)
    norb = nmo
    ham = hamiltonian.hamiltonian(nmo)
    ham_data = {}
    ham_data["h0"] = h0
    ham_data["ene0"] = options["ene0"]

    if 'u' not in trial.lower():
        ham_data["h1"] = jnp.array([h1, h1])
        ham_data["h1_mod"] = jnp.array(h1_mod)
        nchol = chol.shape[0]
        ham_data["chol"] = jnp.array(chol.reshape(chol.shape[0], -1))
    elif 'u' in trial.lower():
        ham_data["h1"] = jnp.array(h1)
        ham_data["h1_mod"] = jnp.array(h1_mod)
        nchol = chol[0].shape[0]
        ham_data["chol"] = jnp.array([chol[0].reshape(chol[0].shape[0], -1),
                                    chol[1].reshape(chol[1].shape[0], -1)])

    wave_data = {}
    prjlo = jnp.array(np.load(mo_file)["prjlo"])
    wave_data["prjlo"] = prjlo
    mo_coeff = jnp.array([np.eye(norb),np.eye(norb)])

    if options["trial"] == "rhf":
        trial = wavefunctions.rhf(norb, nelec_sp, n_batch=options["n_batch"])
        wave_data["mo_coeff"] = mo_coeff[0][:, : nelec_sp[0]]
    elif options["trial"] == "uhf":
        trial = wavefunctions.uhf(norb, nelec_sp, n_batch=options["n_batch"])
        wave_data["mo_coeff"] = [
            mo_coeff[0][:, : nelec_sp[0]],
            mo_coeff[1][:, : nelec_sp[1]],
        ]
    elif options["trial"] == "cisd":
        try:
            amplitudes = np.load(amp_file)
            ci1 = jnp.array(amplitudes["ci1"])
            ci2 = jnp.array(amplitudes["ci2"])
            trial_wave_data = {"ci1": ci1, "ci2": ci2, 
                               "mo_coeff": mo_coeff[0][:, : nelec_sp[0]]}
            wave_data.update(trial_wave_data)
            trial = wavefunctions.cisd(norb, nelec_sp, n_batch=options["n_batch"])
        except:
            raise ValueError("Trial specified as cisd, but amplitudes.npz not found.")
    elif options["trial"] == "ucisd":
        try:
            amplitudes = np.load(amp_file)
            ci1a = jnp.array(amplitudes["ci1a"])
            ci1b = jnp.array(amplitudes["ci1b"])
            ci2aa = jnp.array(amplitudes["ci2aa"])
            ci2ab = jnp.array(amplitudes["ci2ab"])
            ci2bb = jnp.array(amplitudes["ci2bb"])
            trial_wave_data = {
                "ci1A": ci1a,
                "ci1B": ci1b,
                "ci2AA": ci2aa,
                "ci2AB": ci2ab,
                "ci2BB": ci2bb,
                "mo_coeff": mo_coeff,
            }
            wave_data.update(trial_wave_data)
            trial = wavefunctions.ucisd(norb, nelec_sp, n_batch=options["n_batch"])
        except:
            raise ValueError("Trial specified as ucisd, but amplitudes.npz not found.")
    elif options["trial"] == "ccsd_pt":
        trial = wavefunctions.ccsd_pt(norb, nelec_sp, n_batch=options["n_batch"])
        amplitudes = np.load(amp_file)
        t1 = jnp.array(amplitudes["t1"])
        t2 = jnp.array(amplitudes["t2"])
        trial_wave_data = {"t1": t1, "t2": t2}
        wave_data.update(trial_wave_data)
        wave_data["mo_coeff"] = mo_coeff[0][:,:nelec_sp[0]]
    elif options["trial"] == "ccsd_pt2":
        trial = wavefunctions.ccsd_pt2(norb, nelec_sp, n_batch=options["n_batch"])
        nocc = nelec_sp[0]
        amplitudes = np.load(amp_file)
        t1 = jnp.array(amplitudes["t1"])
        t2 = jnp.array(amplitudes["t2"])
        trial_wave_data = {"t1": t1, "t2": t2}
        wave_data.update(trial_wave_data)
        mo_t = trial.thouless_trans(t1)[:,:nocc]
        wave_data['mo_t'] = mo_t
        wave_data["mo_coeff"] = mo_coeff[0][:,:nelec_sp[0]]
    elif options["trial"] == "uccsd_pt":
        trial = wavefunctions.uccsd_pt(norb, nelec_sp, n_batch = options["n_batch"])
        noccA, noccB = trial.nelec[0], trial.nelec[1]
        wave_data["mo_coeff"] = [
            mo_coeff[0][:, : noccA],
            mo_coeff[1][:, : noccB],
        ]
        ham_data['h1_mod'] = h1_mod
        amplitudes = np.load(amp_file)
        t1a = jnp.array(amplitudes["t1a"])
        t1b = jnp.array(amplitudes["t1b"])
        t2aa = jnp.array(amplitudes["t2aa"])
        t2ab = jnp.array(amplitudes["t2ab"])
        t2bb = jnp.array(amplitudes["t2bb"])
        wave_data['t1a'] = t1a
        wave_data['t1b'] = t1b
        wave_data["t2aa"] = t2aa
        wave_data["t2bb"] = t2bb
        wave_data["t2ab"] = t2ab
    elif options["trial"] == "uccsd_pt2":
        trial = wavefunctions.uccsd_pt2(norb, nelec_sp, n_batch = options["n_batch"])
        noccA, noccB = trial.nelec[0], trial.nelec[1]
        wave_data["mo_coeff"] = [
            mo_coeff[0][:, : noccA],
            mo_coeff[1][:, : noccB],
        ]
        ham_data['h1_mod'] = h1_mod
        amplitudes = np.load(amp_file)
        t1a = jnp.array(amplitudes["t1a"])
        t1b = jnp.array(amplitudes["t1b"])
        t2aa = jnp.array(amplitudes["t2aa"])
        t2ab = jnp.array(amplitudes["t2ab"])
        t2bb = jnp.array(amplitudes["t2bb"])
        mo_ta = trial.thouless_trans(t1a)[:,:noccA]
        mo_tb = trial.thouless_trans(t1b)[:,:noccB]
        wave_data['mo_ta'] = mo_ta
        wave_data['mo_tb'] = mo_tb
        wave_data["t2aa"] = t2aa
        wave_data["t2bb"] = t2bb
        wave_data["t2ab"] = t2ab

    if options["walker_type"] == "rhf":
        if options["symmetry"]:
            ham_data["mask"] = jnp.where(jnp.abs(ham_data["h1"]) > 1.0e-10, 1.0, 0.0)
        else:
            ham_data["mask"] = jnp.ones(ham_data["h1"].shape)
        prop = propagation.propagator_restricted(
            options["dt"], 
            options["n_walkers"], 
            options["n_exp_terms"],
            options["n_batch"]
        )

    elif options["walker_type"] == "uhf":
        if options["symmetry"]:
            ham_data["mask"] = jnp.where(jnp.abs(ham_data["h1"]) > 1.0e-10, 1.0, 0.0)
        else:
            ham_data["mask"] = jnp.ones(ham_data["h1"].shape)
        if options["free_projection"]:
            prop = propagation.propagator_unrestricted(
                options["dt"],
                options["n_walkers"],
                10,
                n_batch=options["n_batch"],
            )
        else:
            prop = propagation.propagator_unrestricted(
                options["dt"],
                options["n_walkers"],
                n_batch=options["n_batch"],
            )
    if  'pt' in options['trial']:
        if '2' in options['trial']:
            sampler = sampling.sampler_pt2(
                options["n_prop_steps"],
                options["n_ene_blocks"],
                options["n_sr_blocks"],
                options["n_blocks"],
                nchol,)
        else:
            sampler = sampling.sampler_pt(
                options["n_prop_steps"],
                options["n_ene_blocks"],
                options["n_sr_blocks"],
                options["n_blocks"],
                nchol,)
    else:
        sampler = sampling.sampler(
                options["n_prop_steps"],
                options["n_ene_blocks"],
                options["n_sr_blocks"],
                options["n_blocks"],
                nchol,)

    if rank == 0:
        print(f"# norb: {norb}")
        print(f"# nelec: {nelec_sp}")
        print("#")
        for op in options:
            if options[op] is not None:
                print(f"# {op}: {options[op]}")
        print("#")

    return ham_data, ham, prop, trial, wave_data, sampler, options, MPI

In [6]:
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

print = partial(print, flush=True)

ham_data, ham, prop, trial, wave_data, sampler, options, _ = (_prep_afqmc())

seed = options["seed"]
init_walkers = None
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])

# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 14
# nelec: (3, 3)
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 262
# walker_type: rhf
# trial: cisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
#
-4.402495733105614


In [7]:
import jax
from jax import jit, vmap, lax
import opt_einsum as oe

@partial(jit, static_argnums=0)
def _calc_orb_energy_restricted(
    self, walker: jax.Array, ham_data: dict, wave_data: dict
) -> complex:
    ci1, ci2 = wave_data["ci1"], wave_data["ci2"]
    nocc = self.nelec[0]
    prj = ham_data['prj']

    green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
    green_occ = green[:, nocc:].copy()
    greenp = jnp.vstack((green_occ, -jnp.eye(self.norb - nocc)))

    chol = ham_data["chol"].reshape(-1, self.norb, self.norb)
    rot_chol = chol[:, : self.nelec[0], :]
    h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    hg = oe.contract("pj,pj->", h1[:nocc, :], green, backend="jax")

    # 0 body energy
    e0 = ham_data["h0"]

    # 1 body energy
    # ref
    e1_0 = 2 * hg

    # single excitations
    ci1g_orb = oe.contract("ia,ka,ik->", ci1, green_occ, prj, backend="jax")
    e1_1_1_orb = 4 * ci1g_orb * hg # c_ia G_ia G_pg h_pg
    # gpci1 = greenp @ ci1.T
    # ci1_green = gpci1 @ green
    # ci1gp = ci1 @ greenp.T
    hgpg = oe.contract("pq,pa,iq->ia", h1, greenp, green_occ, backend="jax")
    # e1_1_2 = -2 * oe.contract("ij,ij->", h1, ci1_green, backend="jax")
    e1_1_2_orb = -2 * oe.contract("ia,ka,ik->", ci1, hgpg, prj, backend="jax")
    e1_1_orb = e1_1_1_orb + e1_1_2_orb

    # double excitations
    ggg = oe.contract("ia,ia->jb", ci2, green_occ, backend="jax")
    ci2g_c = oe.contract("iajb,ia->jb", ci2, green_occ, backend="jax")
    ci2g_e = oe.contract("iajb,ib->ja", ci2, green_occ, backend="jax")
    ci2_green_c = (greenp @ ci2g_c.T) @ green
    ci2_green_e = (greenp @ ci2g_e.T) @ green
    ci2_green = 2 * ci2_green_c - ci2_green_e
    ci2g = 2 * ci2g_c - ci2g_e
    gci2g = oe.contract("qu,qu->", ci2g, green_occ, backend="jax")
    e1_2_1 = 2 * hg * gci2g
    e1_2_2 = -2 * oe.contract("ij,ij->", h1, ci2_green, backend="jax")
    e1_2 = e1_2_1 + e1_2_2
    e1 = e1_0 + e1_1 + e1_2

    # two body energy
    # ref
    lg = oe.contract("gpj,pj->g", rot_chol, green, backend="jax")
    # lg1 = jnp.einsum("gpj,pk->gjk", rot_chol, green, optimize="optimal")
    lg1 = oe.contract("gpj,qj->gpq", rot_chol, green, backend="jax")
    e2_0_1 = 2 * lg @ lg
    e2_0_2 = -jnp.sum(vmap(lambda x: x * x.T)(lg1))
    e2_0 = e2_0_1 + e2_0_2

    # single excitations
    e2_1_1 = 2 * e2_0 * ci1g
    lci1g = oe.contract("gij,ij->g", chol, ci1_green, backend="jax")
    e2_1_2 = -2 * (lci1g @ lg)

    ci1g1 = ci1 @ green_occ.T
    # e2_1_3 = jnp.einsum("gpq,gpq->", glgpci1, lg1, optimize="optimal")
    e2_1_3_1 = oe.contract("gpq,gqr,rp->", lg1, lg1, ci1g1, backend="jax")
    lci1g = oe.contract("gip,qi->gpq", ham_data["lci1"], green, backend="jax")
    e2_1_3_2 = -oe.contract("gpq,gqp->", lci1g, lg1, backend="jax")
    e2_1_3 = e2_1_3_1 + e2_1_3_2
    e2_1 = e2_1_1 + 2 * (e2_1_2 + e2_1_3)

    # double excitations
    e2_2_1 = e2_0 * gci2g
    lci2g = oe.contract("gij,ij->g", chol, ci2_green, backend="jax")
    e2_2_2_1 = -lci2g @ lg

    # lci2g1 = jnp.einsum("gij,jk->gik", chol, ci2_green, optimize="optimal")
    # lci2_green = jnp.einsum("gpi,ji->gpj", rot_chol, ci2_green, optimize="optimal")
    # e2_2_2_2 = 0.5 * jnp.einsum("gpi,gpi->", gl, lci2_green, optimize="optimal")
    def scanned_fun(carry, x):
        chol_i, rot_chol_i = x
        gl_i = oe.contract("pj,ji->pi", green, chol_i, backend="jax")
        lci2_green_i = oe.contract(
            "pi,ji->pj", rot_chol_i, ci2_green, backend="jax"
        )
        carry[0] += 0.5 * oe.contract(
            "pi,pi->", gl_i, lci2_green_i, backend="jax"
        )
        glgp_i = oe.contract("pi,it->pt", gl_i, greenp, backend="jax")
        l2ci2_1 = oe.contract(
            "pt,qu,ptqu->",
            glgp_i,
            glgp_i,
            ci2,
            backend="jax"
        )
        l2ci2_2 = oe.contract(
            "pu,qt,ptqu->",
            glgp_i,
            glgp_i,
            ci2,
            backend="jax"
        )
        carry[1] += 2 * l2ci2_1 - l2ci2_2
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol, rot_chol))
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)

    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    e2 = e2_0 + e2_1 + e2_2

    # overlap
    overlap_1 = 2 * ci1g
    overlap_2 = gci2g
    overlap = 1.0 + overlap_1 + overlap_2
    return (e1 + e2) / overlap + e0

In [ ]:
from functools import partial
import jax
from jax import numpy as jnp
from ad_afqmc.prop_unrestricted import wavefunctions
from jax import jit, vmap, lax, jvp
import opt_einsum as oe

@jax.jit
def _ci_orb_olp(walker: jax.Array, wave_data: dict) -> complex:
    '''
    <HF|C1+C2|walker> = C_ia^*G_ia+C_iajb^*(G_iaG_jb-G_ibG_ja)
    prj onto orbital i
    '''
    m = ham_data["prjlo"]
    nocc, ci1, ci2 = walker.shape[1], wave_data["ci1"], wave_data["ci2"]
    gf = (walker.dot(jnp.linalg.inv(walker[: walker.shape[1], :]))).T
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2
    o1 = oe.contract("ia,ka,ik->", ci1, gf[:, nocc:],m, backend="jax")
    o2 = 2 * oe.contract("iajb,ka,jb,ik->", ci2, gf[:, nocc:], gf[:, nocc:],m, backend="jax") \
        - oe.contract("iajb,kb,ja,ik->", ci2, gf[:, nocc:], gf[:, nocc:],m, backend="jax")
    olp = (2*o1+o2)*o0
    return olp

@jax.jit
def hf_ci_walker_olp_r(walker,wave_data):
    '''<HF|walker>/<CISD|walker>'''
    ci1, ci2 = wave_data["ci1"], wave_data["ci2"]
    nocc = walker.shape[1]
    GF = (walker.dot(jnp.linalg.inv(walker[: nocc, :]))).T
    o1 = oe.contract("ia,ia", ci1, GF[:, nocc:], backend="jax")
    o2 = 2 * oe.contract("iajb, ia, jb", ci2, GF[:, nocc:], GF[:, nocc:], backend="jax") \
          - oe.contract("iajb, ib, ja", ci2, GF[:, nocc:], GF[:, nocc:], backend="jax")
    olp_r = 1/(1.0 + 2 * o1 + o2)
    return olp_r


@jax.jit
def _ci_orb_olp1(x: float, h1_mod: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    <HF|(C1+C2)_i exp(x*h1_mod)|walker>
    '''
    walker_1x = walker + x*h1_mod.dot(walker)
    olp = _ci_orb_olp(walker_1x, wave_data)
    return olp

@jax.jit
def _ci_orb_olp2(x: float, chol_i: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    <HF|(C1+C2)_i exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _ci_orb_olp(walker_2x, wave_data)
    return olp

@partial(jit, static_argnums=(3,))
def _hf_eorb(walker, ham_data, wave_data, trial):
    '''hf orbital correlation energy'''
    # <HF|H_i|walker>/<HF|walker>
    rot_h1, rot_chol = ham_data['rot_h1'], ham_data['rot_chol']
    m = ham_data["prjlo"]
    nocc = rot_h1.shape[0]
    _calc_green = wavefunctions.rhf(
        trial.norb,trial.nelec,n_batch=trial.n_batch)._calc_green
    green_walker = _calc_green(walker, wave_data)
    f = oe.contract('gij,jk->gik', rot_chol[:,:nocc,nocc:],
                    green_walker.T[nocc:,:nocc], backend="jax")
    c = vmap(jnp.trace)(f)
    eneo2Jt = oe.contract('Gxk,xk,G->',f,m,c, backend="jax")*2 
    eneo2ext = oe.contract('Gxy,Gyk,xk->',f,f,m, backend="jax")
    hf_orb_en = eneo2Jt - eneo2ext
    return jnp.real(hf_orb_en)


@partial(jit, static_argnums=(3,))
def hf_eorb(walkers,ham_data,wave_data,trial):
    eorb = vmap(_hf_eorb, in_axes=(0, None, None, None))(
        walkers, ham_data, wave_data, trial)
    return eorb

@partial(jit, static_argnums=3)
def _d2_olp2_i(chol_i,walker,wave_data,trial):
    x = 0.0
    f = lambda a: _ci_orb_olp2(a,chol_i,walker,wave_data)
    _, d2f = jax.jvp(lambda x: jax.jvp(f, [x], [1.0])[1], [x], [1.0])
    return d2f

# @partial(jit, static_argnums=3)
# def _d2_olp2(walker,ham_data,wave_data,trial):
#     norb = trial.norb
#     chol = ham_data["chol"].reshape(-1, norb, norb)
#     d2_exp2_batch = jax.vmap(_d2_olp2_i, in_axes=(0,None,None,None))
#     d2_exp2s = d2_exp2_batch(chol,walker,wave_data,trial)
#     h2 = jnp.sum(d2_exp2s)/2
#     return h2

@partial(jit, static_argnums=(3,))
def _ci_eorb(
    walker: jax.Array,
    ham_data: dict,
    wave_data: dict,
    trial: wavefunctions,
):
    '''
    <HF|(C1+C2)_i (H-E0)|walker>/<HF|walker>
    '''

    norb = trial.norb
    chol = ham_data["chol"].reshape(-1, norb, norb)
    h1_mod = ham_data['h1_mod']
    h0_E0 = ham_data["h0"]-ham_data["E0"]

    nocc = walker.shape[1]
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2

    x = 0.0
    # one body
    f1 = lambda a: _ci_orb_olp1(a,h1_mod,walker,wave_data)
    ci_olp, d_overlap = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        walker, wave_data = carry
        return carry, _d2_olp2_i(c,walker,wave_data,trial)

    _, d2_olp2_i = lax.scan(scanned_fun, (walker, wave_data), chol)
    d_2_overlap = jnp.sum(d2_olp2_i)/2

    eorb = (h0_E0*ci_olp + d_overlap + d_2_overlap) / o0

    return jnp.real(eorb)

@partial(jit, static_argnums=(3,))
def _orb_energy(
        walker: jax.Array,
        ham_data: dict,
        wave_data: dict,
        trial: wavefunctions) -> jax.Array:
    
    e_orb = _hf_eorb(walker, ham_data, wave_data, trial) \
         + _ci_eorb(walker, ham_data, wave_data, trial)
    
    return e_orb

In [9]:
from ad_afqmc.prop_unrestricted import wavefunctions
ham_data = wavefunctions.rhf(trial.norb,trial.nelec,n_batch=trial.n_batch
                  )._build_measurement_intermediates(ham_data,wave_data)
ham_data['prjlo'] = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
ham_data["E0"] = mf.e_tot

In [13]:
print(_orb_energy(walker, ham_data, wave_data, trial))

-0.0440622547901864


In [11]:
walker = prop_data['walkers'][0]
eorb0 = _hf_eorb(walker, ham_data, wave_data, trial)
eorb12 = _ci_eorb(walker, ham_data, wave_data, trial)
olp_r = hf_ci_walker_olp_r(walker,wave_data)
print((eorb0+eorb12)*olp_r)

(-0.0440622547901864+0j)


In [12]:
ecorr_cc

'-0.04406226'